# Simple AC Detector

This notebook contains code that will collect data from the microphone and the environmental sensor to try to determine if an air conditioner has been turned on.

### Sound as an Indicator

A baseline volume level is determined during the calibration process. Afterwards, the current volume of the room is compared to this baseline value. If the volume has significantly increased then it will assume the AC has been turned on. This method is very simple be not very accurate. It is easily susceptible to false positives as it attributes any increase in volume to an AC.

### Temperature as an Indicator

A baseline temperature is determined during the calibration process. Afterwards, the current temperature of the room is compared to this baseline value. If the temperature has significantly decreased then it will assume the AC has been turned on. 

The methods used to determine if an AC has been turned on are very simple but not very accurate. To improve upon this methodology, the data from both sensors can be combined to make a better guess. Additionally, the audio data could undergo a more complex analysis process, such as matching the current sound in the room to a known AC sound.

---

## The Code

### Setup

In [1]:
from waggle.data.audio import Microphone
import time
from scipy.io import wavfile
import scipy.io
import matplotlib.pyplot as plt
import numpy as np
import bme680

using backwards compatible implementation of time_ns


In [2]:
# Is the AC on based on the mic
micAC = False
microphone = Microphone()

# Is the AC on based on the envi sensor
enviAC = False
sensor = bme680.BME680()
sensor.set_humidity_oversample(bme680.OS_2X)
sensor.set_pressure_oversample(bme680.OS_4X)
sensor.set_temperature_oversample(bme680.OS_8X)
sensor.set_filter(bme680.FILTER_SIZE_3)
sensor.set_gas_status(bme680.DISABLE_GAS_MEAS)
sensor.set_gas_heater_temperature(320)
sensor.set_gas_heater_duration(150)
sensor.select_gas_heater_profile(0)

## Helper Functions

Records an audio sample for the given duration of time and returns the recording's data.

In [3]:
def getAudioSample(duration):
    sample = microphone.record(duration)
    sample.save("ACDetector.wav")
    (samplerate, data) = wavfile.read('ACDetector.wav')
    return data


Collects temperature readings once a second for the given duration and returns the readings in an array.

In [4]:
def getTempSample(duration):
    enviData = []
    
    for i in range(1, duration):
        if sensor.get_sensor_data():
            reading = sensor.data.temperature
            enviData = np.append(enviData, [reading], axis=0)
            
        time.sleep(1)
        
    return enviData

## Calibration

The calibration part of this notebook should only be run when the AC known to be off.

In [5]:
print("Calibrating Microphone...")
audioData = getAudioSample(10)
noAC = np.average(np.absolute(audioData))
print("Amplitude with AC off: ", noAC)

print("Calibrating Envi Sensor...")
enviData = getTempSample(10)
norm_temp = np.average(enviData)
print("Normal Temperature: ", norm_temp)

Calibrating Microphone...


KeyboardInterrupt: 

## Detect an AC

This loop will alternate between using the microphone and the environmental sensor to try to determine if an AC has been turned on in the last 10 seconds.

In [6]:
while True:
    # Recording Audio
    audioData = getAudioSample(10)
    avgAmp = np.average(np.absolute(audioData))
    print("Currnt avg Amplitude: ", avgAmp)
    print(avgAmp)

    # Check if AC is on based on sound
    if (avgAmp - noAC > 100) and not micAC:
        print("AC is now on based on mic")
        micAC = True  
    elif micAC:
        print("AC is now off based on mic")
        micAC = False

    # Collecting Enviornmental Data
    enviData = getTempSample(10)
    avgTemp = np.average(enviData)
    print("Current avg temp:", avgTemp)
    
    # Check if AC is on based on temperature
    if (norm_temp - avgTemp > 2) and not enviAC:
        print("AC is now on based on envi")
        enviAC = True  
    elif enviAC:
        print("AC is now off based on envi")
        enviAC = False

KeyboardInterrupt: 